In [1]:
import random
import numpy as np
import math
import pickle

In [2]:
class GeneticAlgorithm:
    
    def __init__(self, n_cities, n_paths, mutation_probability):
        self.n_cities = n_cities
        self.n_paths = n_paths
        self.mutation_probability = mutation_probability
         #set up initial population matrix n_paths*(n_cities-1) since we start from city 0
        self.population = np.outer(np.ones(n_paths), np.arange(1,n_cities))
        self.population = np.apply_along_axis(np.random.permutation, 1, self.population)
        
    def build_children(self,v1,v2):
        intersection = np.intersect1d(v1,v2)
        all_cities = np.arange(1,self.n_cities)
        # se v1 e v2 non hanno citta in comune
        if (len(intersection) > 0):
            diff = np.setdiff1d(v1, intersection)
            v1 = [o for o in v1 if o in diff]
        excluded = np.concatenate((v1,v2))
        third = np.setdiff1d(all_cities, excluded)
        return np.concatenate((v1,third,v2))
    
    def distance (self,x1, y1, x2, y2):
        return math.sqrt((x1-x2)**2 + (y1-y2)**2)
        
    def set_cities(self):
        self.x_coord = np.random.uniform(0,100,self.n_cities)
        self.y_coord = np.random.uniform(10,30,self.n_cities)
        self.distances = np.zeros((self.n_cities,self.n_cities))
        for i in range(self.n_cities):
            for j in range(self.n_cities):
                self.distances[i,j] = self.distance( self.x_coord[i],  self.y_coord[i],  self.x_coord[j],  self.y_coord[j])
     
    def initialize_cities(self,x_coord,y_coord):
        self.x_coord = x_coord
        self.y_coord = y_coord
        self.distances = np.zeros((self.n_cities,self.n_cities))
        for i in range(self.n_cities):
            for j in range(self.n_cities):
                self.distances[i,j] = self.distance( self.x_coord[i],  self.y_coord[i],  self.x_coord[j],  self.y_coord[j])
     
    def compute_fitness(self):
        fit_values = np.ones(self.n_paths)
        pos = 0
        for path in self.population:
            l = 0
            for i in range(self.n_cities-2):
                l = l + self.distances[int(path[i]),int(path[i+1])]
            l = l + self.distances[0,int(path[0])]
            l = l + self.distances[int(path[self.n_cities-2]),0]
            fit_values[pos] = l  
            pos = pos + 1
        return np.argsort(fit_values)
    
    #selection function
    # ordina i cammini in base alla fitnes
    # genera un numero p tra 0 e n_paths
    # scegli i primi p cammini
    def selection(self):
        indices = self.compute_fitness()
        p = np.random.randint(2,self.n_paths,1)
        i = 0
        self.fittest_parents = np.outer(np.ones(p), np.arange(1,self.n_cities))
        while(i < p):
            self.fittest_parents[i] = self.population[indices[i]]
            i = i+1
        #print("fittest parents: ")
        #print(self.fittest_parents)

    # dati i primi x migliori genitori
    # per ogni cammino (n_paths)
    # scelgo a caso due genitori
    # scelgo a caso due punti in cui spezzare
    # devo assicurarmi che sia un cammino corretto
    def crossbreeding(self):
        n_parents = len(self.fittest_parents)
        #print("n_parents %s ", n_parents )
        for i in range(self.n_paths-1):
            parent1 = np.random.randint(0,n_parents,1)[0]
            parent2 = np.random.randint(0,n_parents,1)[0]
            #print("p1 %s p2 %s ", parent1, parent2 )
            if (parent1 != parent2) :
                pos1 = np.random.randint(1,self.n_cities-1,1)[0]
                pos2 = np.random.randint(1,self.n_cities-1,1)[0]
                #print("posizioni in cui dividere")
                #print(pos1,pos2)
                parent1 = self.fittest_parents[parent1]
                parent2 = self.fittest_parents[parent2]
                #print(parent1, parent2)
                self.population[i] = self.build_children(parent1[0:pos1], parent2[pos2:])
                self.population[i+1] = self.build_children(parent1[pos2:], parent2[0:pos1])
                #print("population obtained")
                #print( self.population[i],  self.population[i+1])
        
                
    def mutate(self):
        p = np.random.uniform(0,1,1)[0]
        if (p < self.mutation_probability):
            for i in range(self.n_paths):
                path = self.population[i]
                pos1 = np.random.randint(0,self.n_cities-1,1)[0]
                pos2 = np.random.randint(0,self.n_cities-1,1)[0]
                path[[pos1, pos2]] = path[[pos2, pos1]]
                self.population[i] = path
    
    def evolve(self):
        self.selection()
        self.crossbreeding()
        self.mutate()
    
    def best_distance(self):
        fit_values = np.ones(self.n_paths)
        pos = 0
        for path in self.population:
            l = 0
            for i in range(self.n_cities-2):
                l = l + self.distances[int(path[i]),int(path[i+1])]
            l = l + self.distances[0,int(path[0])]
            l = l + self.distances[int(path[self.n_cities-2]),0]
            fit_values[pos] = l  
            pos = pos + 1
        return np.min(fit_values)

In [3]:
def acceptance(cost_starting_state, cost_new_state, beta):
    if cost_new_state <= cost_starting_state:
        return True
    else:
        threshold = np.random.uniform(0,1,1)[0]
        p = np.exp(- (cost_new_state - cost_starting_state)*beta)
        return np.min(np.array([1,p])) > threshold

In [4]:
with open('x.pickle', 'rb') as handle:
    x_coord = pickle.load(handle)
with open('y.pickle', 'rb') as handle:
    y_coord = pickle.load(handle)

In [6]:
ga = GeneticAlgorithm(len(x_coord),40, 0.01)
ga.initialize_cities(x_coord,y_coord)
betas = np.arange(1,1500,100)
n_iterations = 50
for beta in betas:
    for i in range(n_iterations):
        cost_starting_state = ga.compute_fitness()
        starting_state = ga.population
        ga.evolve()
        cost_new_state = ga.compute_fitness()
        for i in range(ga.n_paths):
            if not acceptance(cost_starting_state[i], cost_new_state[i], beta):
                  ga.population[i] = starting_state[i]
        print(ga.best_distance())

8.257857155353635
7.634906911262542
7.634906911262542
7.64586406079404
7.338164187215707
6.884656772636604
6.884656772636604
7.084123967402974
6.523178323770271
6.523178323770271
6.289143123537776
6.289143123537776
6.289143123537776
6.289143123537776
6.289143123537776
7.033500737823715
7.042310634062001
7.457334489285675
7.5552615108715875
7.635773591151213
7.635773591151213
7.635773591151213
7.635773591151213
7.635773591151213
7.573705492977172
7.635773591151213
7.471027578578396
7.471027578578396
7.408959480404356
7.471027578578396
7.471027578578396
7.408959480404356
7.408959480404356
6.965848579911376
6.95168072385323
6.95168072385323
6.95168072385323
6.6651268655684826
6.965848579911376
6.903780481737336
6.903780481737336
6.965848579911376
6.679294721626628
6.679294721626628
6.679294721626628
6.679294721626628
6.914876555606924
6.941671993125857
6.980396645999991
6.980396645999991
6.914876555606924
6.918328547825951
6.980396645999991
6.964905451196106
6.6651268655684826
6.665126865

6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.370393000949694
6.37039300